#### Importar as Bibliotecas necessárias

In [1]:
import pandas as pd
from pulp import LpProblem, LpVariable, lpSum, LpMinimize, LpStatus

#### Leitura dos arquivos de entrada e conversão para dicionários

In [5]:
# Ler e converter custos
df_custos_loaded = pd.read_excel(r'input/custos.xlsx')
custos = {(row['Fabrica'], row['Produto']): row['Custo'] for _, row in df_custos_loaded.iterrows()}

# Ler e converter custos de frete
df_custo_frete_loaded = pd.read_excel(r'input/custo_frete.xlsx')
custo_frete = {(row['Fabrica'], row['Cliente']): row['Custo Frete'] for _, row in df_custo_frete_loaded.iterrows()}

# Ler e converter capacidades das fábricas
df_capacidade_fabricas_loaded = pd.read_excel(r'input/capacidade_fabricas.xlsx')
capacidade_fabricas = {row['Fabrica']: row['Capacidade'] for _, row in df_capacidade_fabricas_loaded.iterrows()}

# Ler e converter demandas dos clientes
df_demanda_clientes_loaded = pd.read_excel(r'input/demanda_clientes.xlsx')
demanda_clientes = {}
for _, row in df_demanda_clientes_loaded.iterrows():
    cliente = row['Cliente']
    produto = row['Produto']
    demanda = row['Demanda']
    if cliente not in demanda_clientes:
        demanda_clientes[cliente] = {}
    demanda_clientes[cliente][produto] = demanda

# Ler e converter rendimento
df_rendimento_loaded = pd.read_excel(r'input/rendimento.xlsx')
rendimento = {(row['Fabrica'], row['Produto']): row['Rendimento'] for _, row in df_rendimento_loaded.iterrows()}

# Ler e converter lote mínimo
df_lote_minimo_loaded = pd.read_excel(r'input/lote_minimo.xlsx')
lote_minimo = {(row['Fabrica'], row['Produto']): row['Lote Minimo Produção'] for _, row in df_lote_minimo_loaded.iterrows()}

# Fabricas
fabricas = list(df_custos_loaded['Fabrica'].unique())

# Produtos
produtos = list(df_demanda_clientes_loaded['Produto'].unique())

# Clientes
clientes = list(df_demanda_clientes_loaded['Cliente'].unique())

#### Etapa que verifica se todos os arquivos de inputs estão preenchidos com todas as chaves necessárias

In [36]:
# Função para verificar se todos os clientes de demanda_clientes estão em custo_frete
def verificar_clientes(demanda_clientes, custo_frete):
    clientes_demanda = set(demanda_clientes.keys())
    clientes_frete = set(cliente for _, cliente in custo_frete.keys())
    
    clientes_faltantes = clientes_demanda - clientes_frete
    
    if clientes_faltantes:
        print(f"Clientes faltando em custo_frete: {clientes_faltantes}")
    else:
        print("Todos os clientes de demanda_clientes estão presentes em custo_frete.")

# Função para verificar se todos os produtos de demanda_clientes estão em custos e lote_minimo
def verificar_produtos(demanda_clientes, custos, lote_minimo):
    produtos_demanda = {produto for produtos in demanda_clientes.values() for produto in produtos.keys()}
    
    produtos_custos = set(produto for _, produto in custos.keys())
    produtos_lote_minimo = set(produto for _, produto in lote_minimo.keys())
    
    produtos_faltantes_custos = produtos_demanda - produtos_custos
    produtos_faltantes_lote_minimo = produtos_demanda - produtos_lote_minimo
    
    if produtos_faltantes_custos:
        print(f"Produtos faltando em custos: {produtos_faltantes_custos}")
    else:
        print("Todos os produtos de demanda_clientes estão presentes em custos.")
    
    if produtos_faltantes_lote_minimo:
        print(f"Produtos faltando em lote_minimo: {produtos_faltantes_lote_minimo}")
    else:
        print("Todos os produtos de demanda_clientes estão presentes em lote_minimo.")

# Executar as verificações
verificar_clientes(demanda_clientes, custo_frete)
verificar_produtos(demanda_clientes, custos, lote_minimo)

Todos os clientes de demanda_clientes estão presentes em custo_frete.
Todos os produtos de demanda_clientes estão presentes em custos.
Todos os produtos de demanda_clientes estão presentes em lote_minimo.


#### Modelo de otimização olhando a minimização dos custos totais (Custo de Produto + Frete)

In [7]:
# Criando o problema
problema = LpProblem("Network_Design", LpMinimize)

# Definindo as variáveis de decisão
x = LpVariable.dicts("Producao", [(fabrica, produto, cliente) for fabrica in fabricas for produto in produtos for cliente in clientes], lowBound=0, cat="Continuous")

# Definindo as variáveis de lote mínimo
lote_minimo_vars = LpVariable.dicts("LoteMinimo", [(fabrica, produto) for fabrica in fabricas for produto in produtos], lowBound=0, cat="Continuous")

# Definindo a função objetivo para minimizar o custo total, incluindo o custo de frete
problema += (lpSum(x[(fabrica, produto, cliente)] * custos[(fabrica, produto)] for fabrica in fabricas for produto in produtos for cliente in clientes) 
            + lpSum(x[(fabrica, produto, cliente)] * custo_frete[(fabrica, cliente)] for fabrica in fabricas for produto in produtos for cliente in clientes)), "Minimize_Total_Cost"

# Restrições de produção para cada fábrica
for fabrica in fabricas:
    problema += lpSum(x[(fabrica, produto, cliente)] for produto in produtos for cliente in clientes) <= capacidade_fabricas[fabrica], f"{fabrica}_Capacity"

# Restrições de demanda para cada cliente
for cliente in clientes:
    for produto in produtos:
        if produto in demanda_clientes[cliente]:  # Verifica se o produto existe para o cliente
            problema += lpSum(x[(fabrica, produto, cliente)] for fabrica in fabricas) >= demanda_clientes[cliente][produto] + ((1 - rendimento.get((fabrica, produto), 1.0)) * demanda_clientes[cliente][produto]), f"{cliente}_Demand_{produto}"

# Restrições de lote mínimo para cada fábrica e produto
for fabrica in fabricas:
    for produto in produtos:
        problema += lpSum(x[(fabrica, produto, cliente)] for cliente in clientes) >= lote_minimo[(fabrica, produto)], f"{fabrica}_{produto}_LoteMinimo"

# Resolvendo o problema
problema.solve()

# Exibindo resultados
print("Status da otimização:", LpStatus[problema.status])

# Criando uma lista para armazenar os resultados
resultados_lista = []

# Preenchendo a lista com os resultados
for fabrica in fabricas:
    for produto in produtos:
        for cliente in clientes:
            quantidade = x[(fabrica, produto, cliente)].varValue
            custo_unitario = custos[(fabrica, produto)]
            custo_frete_ = custo_frete[(fabrica, cliente)]
            custo_total = quantidade * (custo_unitario + custo_frete_)
            resultados_lista.append({
                "Fabrica": fabrica,
                "Produto": produto,
                "Cliente": cliente,
                "Quantidade": round(quantidade, 2),
                "Custo Unitário": round(custo_unitario, 2),
                "Custo Frete": round(custo_frete_, 2),
                "Custo Total": round(custo_total, 2)
            })

# Criando o DataFrame a partir da lista de resultados
resultados = pd.DataFrame(resultados_lista)

# Salvando os resultados detalhados em uma planilha de excel na pasta output
resultados.to_excel(r'output/resultados_detalhados.xlsx', index=False)

# Print de valores produzidos por fábrica
print("\nCusto total por fábrica:")
for fabrica in fabricas:
    custo_total_fabrica = sum(x[(fabrica, produto, cliente)].varValue * (custos[(fabrica, produto)] + custo_frete[(fabrica, cliente)]) for produto in produtos for cliente in clientes)
    print(f"{fabrica}: {round(custo_total_fabrica, 2)}")

# Print do total produzido por fábrica
print("\nTotal Otimizado produzido por fábrica:")
for fabrica in fabricas:
    total_produzido_fabrica = sum(x[(fabrica, produto, cliente)].varValue for produto in produtos for cliente in clientes)
    print(f"{fabrica}: {round(total_produzido_fabrica, 2)}")

# Exibindo o DataFrame de resultados
print("\nResultados Detalhados:")
resultados

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/vitortatekawa/Documents/04.Projetos/NetworkDesignOptimization/lib/python3.10/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/wy/4_8wnj8d5x9_cjytnn5333yh0000gn/T/6825622604e841c5a25be6b5e135ee7e-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/wy/4_8wnj8d5x9_cjytnn5333yh0000gn/T/6825622604e841c5a25be6b5e135ee7e-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 6985 COLUMNS
At line 666686 RHS
At line 673667 BOUNDS
At line 673668 ENDATA
Problem MODEL has 6980 rows, 200000 columns and 459700 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 6980 (0) rows, 60700 (-139300) columns and 181100 (-278600) elements
0  Obj 0 Primal inf 196061.46 (6970)
214  Obj 323796.12 Primal inf 181382.07 (6662)
428  Obj 619429.56 Primal inf 168752.24 (6401)
642  Obj 899986.06 Primal inf 156909.

,Fabrica,Produto,Cliente,Quantidade,Custo Unitário,Custo Frete,Custo Total
0,Fabrica_1,Produto_7,Cliente_1,15.87,15.23,4.52,313.43
1,Fabrica_1,Produto_7,Cliente_2,0.00,15.23,11.30,0.00
2,Fabrica_1,Produto_7,Cliente_3,0.00,15.23,17.59,0.00
3,Fabrica_1,Produto_7,Cliente_4,0.00,15.23,14.91,0.00
4,Fabrica_1,Produto_7,Cliente_5,0.00,15.23,16.32,0.00
...,...,...,...,...,...,...,...
199995,Fabrica_10,Produto_18,Cliente_196,0.00,95.36,17.44,0.00
199996,Fabrica_10,Produto_18,Cliente_197,0.00,95.36,3.99,0.00
199997,Fabrica_10,Produto_18,Cliente_198,0.00,95.36,6.89,0.00
199998,Fabrica_10,Produto_18,Cliente_199,0.00,95.36,6.51,0.00
